# Generate Final Results
This notebook I will use to save each cost type, each agency correction, and the final numbers to a .csv. These .csv's we will use for visualizing results and checking code. The code here will also be the basis of my test code for refactoring.

## Set up Environment

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import os
import sys
import numpy as np
from sodapy import Socrata

In [4]:
sys.path.insert(0, os.path.abspath("../"))

In [8]:
from Initialize_Agencies_3Ver import get_agencies
yr = list(range(2016,2020))
agencies = get_agencies(yr)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory/pensions/State_Pensions_2Ver.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [130]:
final_results = pd.read_csv("Final_by_Agency_Type_Dec27.csv", index_col = 0)
final_results.rename(columns = {x:int(x) for x in final_results.columns[2:]}, inplace=True)

In [155]:
final_results#.loc["trial_court Non-Payroll Operating"]

Agency              Cost Type  \
trial_court Payroll Costs                trial_court                Payroll   
trial_court Non-Payroll Operating Costs  trial_court  Non-Payroll Operating   
trial_court Pension Costs                trial_court               Pensions   
trial_court Fringe Benefit Costs         trial_court        Fringe Benefits   
trial_court Capital Costs                trial_court                Capital   
...                                              ...                    ...   
Winthrop PD Payroll Costs                Winthrop PD                Payroll   
Winthrop PD Non-Payroll Operating Costs  Winthrop PD  Non-Payroll Operating   
Winthrop PD Pension Costs                Winthrop PD               Pensions   
Winthrop PD Fringe Benefit Costs         Winthrop PD        Fringe Benefits   
Winthrop PD Capital Costs                Winthrop PD                Capital   

                                                2016         2017  \
trial_court Payroll Costs               23972722.911 22938605.953   
trial_court Non-Payroll Operating Costs  5459896.759  5063328.750   
trial_court Pension Costs                3568017.523  3772546.408   
trial_court Fringe Benefit Costs         9601913.337  9130258.367   
trial_court Capital Costs                3687547.641  3166614.121   
...                                              ...          ...   
Winthrop PD Payroll Costs                2842909.000  2927191.000   
Winthrop PD Non-Payroll Operating Costs   194777.000   185439.000   
Winthrop PD Pension Costs                 912028.782   702707.081   
Winthrop PD Fringe Benefit Costs          794997.427  1188270.591   
Winthrop PD Capital Costs                      0.000    39393.000   

                                                2018         2019  
trial_court Payroll Costs               25815412.064 28271447.043  
trial_court Non-Payroll Operating Costs  5631905.902  6725293.128  
trial_court Pension Costs                4348113.459  4929516.703  
trial_court Fringe Benefit Costs         9596799.087 10122247.192  
trial_court Capital Costs                3658071.571  3719111.121  
...                                              ...          ...  
Winthrop PD Payroll Costs                3066911.000  3803402.000  
Winthrop PD Non-Payroll Operating Costs   224594.000   233533.000  
Winthrop PD Pension Costs                 745260.916   748358.573  
Winthrop PD Fringe Benefit Costs         1108502.513  1116049.008  
Winthrop PD Capital Costs                  40000.000        0.000  

[85 rows x 6 columns]

In [151]:
trial_court = agencies["trial_court"]
trial_court.__dict__["non_payroll_operating_expenditures_by_year"]

budget_fiscal_year
2016   111466652.300
2017   108356313.990
2018   110473319.370
2019   124205832.610
Name: amount, dtype: float64

# Check

In [169]:
index_attr_dict = {" Payroll Costs": "payroll_by_year",
                   " Non-Payroll Operating Costs": "non_payroll_operating_expenditures_by_year",
                   " Pension Costs": "pensions",
                   " Fringe Benefit Costs": "fringe",
                   " Capital Costs": "capital_expenditures_by_year"}
for _, agency in agencies.items():
    for cti, cta in index_attr_dict.items():
        true = final_results.loc[agency.alias + cti , yr].astype(float).values
        current = agency.correction_function(agency.__dict__[cta]).astype(float).values 
        assert np.allclose(a=true, b=current, rtol=1e-10), \
             cti + " for "+ agency.alias+ " not equal"
print("all passed!!")

#     final_by_agency_type.loc[agency.alias + " Payroll Costs"] = \
#      [agency.alias]  + ["Payroll"] + list(agency.correction_function(agency.payroll_by_year))
#     final_by_agency_type.loc[agency.alias + " Non-Payroll Operating Costs"] = \
#      [agency.alias] + ["Non-Payroll Operating"] +\
#      list(agency.correction_function(agency.non_payroll_operating_expenditures_by_year))
#     final_by_agency_type.loc[agency.alias + " Pension Costs"] =  \
#      [agency.alias] + ["Pensions"] + list(agency.correction_function(agency.pensions))
#     final_by_agency_type.loc[agency.alias + " Fringe Benefit Costs"] = \
#      [agency.alias] + ["Fringe Benefits"] + list(agency.correction_function(agency.fringe))
#     final_by_agency_type.loc[agency.alias + " Capital Costs"] = \
#      [agency.alias] + ["Capital"] + list(agency.correction_function(agency.capital_expenditures_by_year))

all passed!!
